In [1]:
from glob import glob
import xml.etree.ElementTree as ET

import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import load_model


In [2]:
images = []
original_images = []
annotations = []

for img, annotation in zip(glob('../../dataset/images/*.png'), glob('../../dataset/annotations/*.xml')):
    image = cv2.imread(img)
    original_images.append(image)

    xml = ET.parse(annotation)
    info = xml.getroot().find('object').find('bndbox')
    xmin = int(info.find('xmin').text)
    ymin = int(info.find('ymin').text)
    xmax = int(info.find('xmax').text)
    ymax = int(info.find('ymax').text)

    shape = image.shape
    xmin = xmin / shape[1]
    xmax = xmax / shape[1]
    ymin = ymin / shape[0]
    ymax = ymax / shape[0]

    image = cv2.resize(image, (224, 224))

    images.append(image)
    annotations.append((xmin, ymin, xmax, ymax))

images = np.array(images) / 255
annotations = np.array(annotations)


In [3]:
model = load_model('./model_50.h5')
x = images[int(0.9 * images.shape[0]):]
y_pred = model.predict(x)
y_test = list(annotations)
original_x_test = original_images[int(0.9 * images.shape[0]):]


c:\Users\Stefan\anaconda3\envs\tf\lib\site-packages\keras\layers\core\lambda_layer.py:303: UserWarning: keras.applications.inception_resnet_v2 is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(config, custom_objects,


2/2 [==============================] - 6s 520ms/step


In [4]:
for i, a in enumerate(zip(y_pred, y_test)):
    x, y = a

    xmin, ymin, xmax, ymax = x[0], x[1], x[2], x[3]
    xmin, xmax = int(xmin * original_x_test[i].shape[1]), int(xmax * original_x_test[i].shape[1])
    ymin, ymax = int(ymin * original_x_test[i].shape[0]), int(ymax * original_x_test[i].shape[0])

    box = cv2.rectangle(original_x_test[i], (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

    if min(box.shape) == 0:
        print(i)
        continue

    cv2.imwrite(f'./results/image_{i}.png', box)
